In [1]:
print("ok")

ok


In [2]:
%pwd

'/Users/niloy/Code/GenerativeAI-Workspace/GenAI-Medical-Chatbot/research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'/Users/niloy/Code/GenerativeAI-Workspace/GenAI-Medical-Chatbot'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
# Extract data from PDF
def load_pdf_file(data):
    loader = DirectoryLoader(data, glob="*.pdf",loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [7]:
extracted_data = load_pdf_file(data="Data/")

In [8]:
#extracted_data

In [9]:
# Split the data into text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [10]:
text_chunks = text_split(extracted_data)
print(f"Number of text chunks: {len(text_chunks)}")

Number of text chunks: 8706


In [11]:
from langchain.embeddings import HuggingFaceEmbeddings

In [12]:
# Download the embeddings from HuggingFace
def download_embeddings():
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )
    return embeddings

In [14]:
embeddings = download_embeddings()

In [15]:
query_result = embeddings.embed_query("Hello")
print(f"Query result length: {len(query_result)}")

Query result length: 384


In [16]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")

Pinecone API Key: pcsk_4uuP9L_7B9Nk72et7QcGXcvbQhb2chFQmVop82wejUuGeDko3oZLRBcX7sqn1QhuLRuo4N


In [18]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "genai-medical-chatbot"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

{
    "name": "genai-medical-chatbot",
    "metric": "cosine",
    "host": "genai-medical-chatbot-8bs34lt.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [19]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [20]:
# Embed each chunk and upsert the embeddings into your Pinecone index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    text_chunks,
    embeddings,
    index_name=index_name,
    namespace="genai-medical-chatbot"
)

In [21]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
    namespace="genai-medical-chatbot"
)

In [22]:
retriever = docsearch.as_retriever(search_type = "similarity",search_kwargs={"k": 3})

In [23]:
retriever_docs = retriever.invoke("What is acne?")

In [24]:
retriever_docs

[Document(id='2788b837-74eb-4e78-925e-75f1e5cac231', metadata={'creationdate': '2004-12-18T17:52:16-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T18:38:56-06:00', 'page': 685.0, 'page_label': '686', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data/Medical_book.pdf', 'total_pages': 940.0}, page_content='rare disorder characterized by an abnormal increase in\nthe number of mature red cells in the blood.\nGALE ENCYCLOPEDIA OF MEDICINE 2 2981\nSecondary polycythemia\nThis young boy is afflicted with seborrheic dermatitis.(Cus-\ntom Medical Stock Photo. Reproduced by permission.)\nKEY TERMS\nAcne—A chronic inflammation of the sebaceous\nglands that manifests as blackheads, whiteheads,\nand/or pustules on the face or trunk.\nPsoriasis—A skin disorder of chronic, itchy scaling'),
 Document(id='9be02841-7bc8-4c41-a605-a22259607923', metadata={'creationdate': '2004-12-18T17:52:16-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T18:38:56-06:00', 'page': 622.0, 'page_label': '623', 

In [43]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(
    model="llama3.2:1b",
    temperature=0.4,
    num_predict=500  # equivalent to max_tokens
)

In [45]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import  create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question answering tasks."
    "Use the following pieces of retrieved context to answer the question."
    "If you don't know the answer, just say that you don't know."
    "Do not try to make up an answer."
    "Use 3 sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])

In [46]:
question_answer_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt,
)

rag_chain = create_retrieval_chain(
    retriever=retriever,
    combine_docs_chain=question_answer_chain,
)

In [47]:
response = rag_chain.invoke({"input": "What is acne?"})
print(response["answer"])

Acne is a skin disorder characterized by an abnormal increase in mature red blood cells in the blood, leading to inflammation and scaling on the face or trunk. It can be mistaken for rosacea due to similar symptoms such as redness and spider-like veins, but blackheads and cysts are not typically seen in patients with acne vulgaris. Treatment options include contraception, topical vitamin A derivatives, isotretinoin, and azelaic acid, which may reduce the severity of symptoms.


In [50]:
response = rag_chain.invoke({"input": "Do you know Niloy?"})
print(response["answer"])

I don't know who Niloy is.
